# Importazione delle librerie

<a target="_blank" href="https://colab.research.google.com/github/socrates77/snn-tre/blob/main/notebooks/Rate-coding-luminosita-di-un-pixel.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [9]:
%pip install snnTorch

Note: you may need to restart the kernel to use updated packages.


In [10]:
import matplotlib
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import io
from PIL import Image
import torch
from snntorch import spikegen

font = {
    'size': 18
}
matplotlib.rc('font', **font)

## Classi per la gestione dei parametri variabili

In [11]:
class Threshold:
    v_threshold=0.09
    def __init__(self, v_threshold=0.09):
        self.v_threshold = v_threshold
    def set_v_threshold(self, v_threshold):
        self.v_threshold = v_threshold
    def get_v_threshold(self):
        return self.v_threshold
    
class PixelIntensity:
    pixel_intensity=0.0
    def set_pixel_intensity(self, pixel_intensity):
        self.pixel_intensity = pixel_intensity;
    def get_pixel_intensity(self):
        return self.pixel_intensity

v_th_obj = Threshold()
pixel_intensity = PixelIntensity()

## Modello e funzione per graficare

In [12]:
output = widgets.Output()

def lif_neuron(v, time_step=1e-3, v_th=35e-3, I=0, R=5e7, C=1e-10):
  '''
  Implementazione del modello Leaky Integrate and Fire in python.
  '''
  tau=R*C
  spk = (v>=v_th)
  # Sottraggo la tensione di soglia quando ho un spike
  v = v + (time_step/tau)*(-v + I*R) - (spk*v_th) 
    
  return v, spk

def v_mem_spike_plot(time_vector, v_mem, spk_rec, I=[], filename='figure.eps', format='eps', title=''):
  # Creare la figura
  spike_record = [1 if element == 1 else None for element in spk_rec]
  fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(14, 9), height_ratios=[10, 1]) # crea la figura e gli assi
  fig.tight_layout()
  axs[0].set_title(title)
  
  if ( len(I) != 0 and len(I) == len(v_mem) ):
    axs_1 = axs[0].twinx()
    axs[0].plot(time_vector, v_mem, scaley=True) # disegna il grafico della tensione ai capi del condensatore
    axs_1.set_ylabel('$I_{syn}$', color='r')
    axs_1.plot(time_vector, I, 'r-', scaley=True)
  else:
    axs[0].plot(time_vector, v_mem) # disegna il grafico della tensione ai capi del condensatore

  axs[0].set_xlim((-2, 202))
  axs[0].set_xlabel('Tempo') # aggiungi l'etichetta all'asse x
  axs[0].set_ylabel('Potenziale di membrana') # aggiungi l'etichetta all'asse y

  axs[1].scatter(time_vector, spike_record, marker='|', color=['b'])
  axs[1].set_yticks([])
  axs[1].set_xlim((-2, 202))
  axs[1].set_xlabel('Tempo') # aggiungi l'etichetta all'asse x
  axs[1].set_ylabel('Spikes') # aggiungi l'etichetta all'asse y

  # Salvare la figura in formato EPS
  fig.savefig(filename, format=format)

  with output:
    output.clear_output(wait=True)
    plt.show()

# Widgets Original

In [21]:
pixel_intensity_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=255,
    step=3,
    description='Pixel Intensity:',
    orientation='vertical',
    readout=True,
    # readout_format='.2f',
)

v_threshold_slider = widgets.FloatSlider(
    value=0.1,
    min=0.09,
    max=1,
    step=0.01,
    description='Threshold:',
    orientation='vertical',
    readout=True,
    readout_format='.2f',
)

# Creiamo un widget Image per mostrare il pixel
pixel_image = widgets.Image(
    format='jpeg',
    width=100,
    height=100,
)

def init_pixel_image():
    pixel_intensity_value=0.0
    image_array = np.ones( (100,100), dtype=np.uint8 )*pixel_intensity_value
    # Convertiamo l'array in un oggetto immagine JPEG
    pil_image = Image.fromarray(image_array)
    pil_image = pil_image.convert('L')
    byte_io = io.BytesIO()
    pil_image.save(byte_io, 'jpeg')
    image_data = byte_io.getvalue()
    pixel_image.value = image_data

def update_pixel_image(change):
    pixel_intensity_value=change.new
    image_array = np.ones( (100,100), dtype=np.uint8 )*pixel_intensity_value
    # Convertiamo l'array in un oggetto immagine JPEG
    pil_image = Image.fromarray(image_array)
    pil_image = pil_image.convert('L')
    byte_io = io.BytesIO()
    pil_image.save(byte_io, 'jpeg')
    image_data = byte_io.getvalue()
    pixel_image.value = image_data

def update_v_threshold(change):
    v_th_obj.set_v_threshold(change.new)
    run_network(pixel_intensity.get_pixel_intensity(), v_th_obj.get_v_threshold())

def update_pixel(change):
    pixel_intensity.set_pixel_intensity(change.new)
    run_network(pixel_intensity.get_pixel_intensity(), v_th_obj.get_v_threshold())

def run_network(pixel_intensity_value, v_threshold):
    spike_vector, _ = spikegen.target_rate_code(num_steps=1250, rate=(pixel_intensity_value/(255*10)))
    spike_vector = spike_vector*1e-8
    prev = 0.0
    spk = 0.0
    spk_rec = torch.zeros_like(spike_vector)
    v_mem = torch.zeros_like(spike_vector)
    time_vector = range(0, len(spike_vector))

    for i in time_vector:
        I = spike_vector[i]
        v_mem[i] = prev
        spk_rec[i] = spk
        prev, spk = lif_neuron(prev, I=I, v_th=v_threshold )
    figure="../figures/rate-coding-spike-intensity-{:.2f}-threshold-{:.2f}.eps".format( pixel_intensity_value, v_threshold )
    title="Rate coding, Pixel Intensity {:.2f}, Threshold: {:.2f}".format( pixel_intensity_value, v_threshold )
    v_mem_spike_plot(time_vector, v_mem, spk_rec, spike_vector, filename=figure, title=title)


In [22]:
init_pixel_image()
run_network(pixel_intensity.get_pixel_intensity(), v_th_obj.get_v_threshold())
v_threshold_slider.observe(update_v_threshold, names='value')
pixel_intensity_slider.observe(update_pixel, names='value')
pixel_intensity_slider.observe(update_pixel_image, names='value')
container = widgets.HBox(children=[ pixel_intensity_slider, v_threshold_slider, pixel_image ] )
display( widgets.VBox(children=[ container, output ] ) )

## Widgets

In [15]:
pixel_intensity_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=255,
    step=3,
    description='Pixel Intensity:',
    orientation='vertical',
    readout=True,
    # readout_format='.2f',
)

v_threshold_slider = widgets.FloatSlider(
    value=0.1,
    min=0.09,
    max=1,
    step=0.01,
    description='Threshold:',
    orientation='vertical',
    readout=True,
    readout_format='.2f',
)

# Creiamo un widget Image per mostrare il pixel
pixel_image = widgets.Image(
    format='jpeg',
    width=100,
    height=100,
)

def init_pixel_image():
    pixel_intensity_value=0.0
    image_array = np.ones( (100,100), dtype=np.uint8 )*pixel_intensity_value
    # Convertiamo l'array in un oggetto immagine JPEG
    pil_image = Image.fromarray(image_array)
    pil_image = pil_image.convert('L')
    byte_io = io.BytesIO()
    pil_image.save(byte_io, 'jpeg')
    image_data = byte_io.getvalue()
    pixel_image.value = image_data

def update_pixel_image(change):
    pixel_intensity_value=change.new
    image_array = np.ones( (100,100), dtype=np.uint8 )*pixel_intensity_value
    # Convertiamo l'array in un oggetto immagine JPEG
    pil_image = Image.fromarray(image_array)
    pil_image = pil_image.convert('L')
    byte_io = io.BytesIO()
    pil_image.save(byte_io, 'jpeg')
    image_data = byte_io.getvalue()
    pixel_image.value = image_data

def update_v_threshold(change):
    v_th_obj.set_v_threshold(change.new)
    run_network(pixel_intensity.get_pixel_intensity(), v_th_obj.get_v_threshold())

def update_pixel(change):
    pixel_intensity.set_pixel_intensity(change.new)
    run_network(pixel_intensity.get_pixel_intensity(), v_th_obj.get_v_threshold())

def run_network(pixel_intensity_value, v_threshold):
    spike_vector = spikegen.rate(num_steps=200, data=torch.tensor((pixel_intensity_value/(255*10))))
    spike_vector = spike_vector*5e-8
    prev = 0.0
    spk = 0.0
    spk_rec = torch.zeros_like(spike_vector)
    v_mem = torch.zeros_like(spike_vector)
    time_vector = range(0, len(spike_vector))

    for i in time_vector:
        I = spike_vector[i]
        v_mem[i] = prev
        spk_rec[i] = spk
        prev, spk = lif_neuron(prev, I=I, v_th=v_threshold )
    figure="../figures/rate-coding-spike-intensity-{:.2f}-threshold-{:.2f}-rate-function.eps".format( pixel_intensity_value, v_threshold )
    title="Rate coding, Pixel Intensity {:.2f}, Threshold: {:.2f}".format( pixel_intensity_value, v_threshold )
    v_mem_spike_plot(time_vector, v_mem, spk_rec, spike_vector, filename=figure, title=title)


In [16]:
init_pixel_image()
run_network(pixel_intensity.get_pixel_intensity(), v_th_obj.get_v_threshold())
v_threshold_slider.observe(update_v_threshold, names='value')
pixel_intensity_slider.observe(update_pixel, names='value')
pixel_intensity_slider.observe(update_pixel_image, names='value')
container = widgets.HBox(children=[ pixel_intensity_slider, v_threshold_slider, pixel_image ] )
display( widgets.VBox(children=[ container, output ] ) )